In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np
import scipy.io.wavfile as wavfile
from io import *
import os.path

In [11]:
# From TFRecord Tutorial
# The following functions can be used to convert a value to a type compatible

def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [12]:
def slice_signal(signal, window_size, stride=0.5):
    """ Return windows of the given signal by sweeping in stride fractions
        of window
    """
    assert signal.ndim == 1, signal.ndim
    n_samples = signal.shape[0]
    offset = int(window_size * stride)
    slices = []
    for beg_i in range(0, n_samples, offset):
        end_i = beg_i + offset
        slice_ = signal[beg_i:end_i]
        # pad the rest of the slice with 0
        if end_i - beg_i > n_samples - beg_i:
            slice_ = signal[beg_i:n_samples]
            test_l = np.zeros((offset-(n_samples-beg_i)))
            slice_ = np.concatenate((slice_,test_l),axis=None)
        if slice_.shape[0] == offset:
            slices.append(slice_)
    return np.array(slices, dtype=np.int32)

In [36]:
def make_tf_example(sequence, label):
    lbl = 0
    if label == 'noisy':
        lbl = 1
    example = tf.train.SequenceExample()
    #features=tf.train.Features(feature={'wav_raw': _int64_feature(audio_serial), 'label': _bytes_feature([lbl])}))
    example.context.feature["label"].int64_list.value.append(lbl)
    # Feature lists for the sequential features
    feat_values = example.feature_lists.feature_list["values"]
    for seq in sequence:
        for value in seq:
            feat_values.feature.add().int64_list.value.append(value)
    return example

In [37]:
# Define the data
DATADIR = r".\Dataset"
LABELS = ["clean", "noisy"]

for lbl in LABELS:
    i = 1
    path = os.path.join(DATADIR, lbl)  # create path to sequences
    save_path = os.path.join(DATADIR,"records",lbl+"_save02.tfrecords")
    for file in os.listdir(path):  # iterate over each image per dogs and cats
        out_file = tf.io.TFRecordWriter(save_path) # instantiate the recorder
        fm, wav_data = wavfile.read(os.path.join(path,file)) # read the wav file
        audio_serial = slice_signal(wav_data, 2 ** 14, stride=1) # transform it into a np.array
        if fm != 16000: # check it sampling
            raise ValueError('Sampling rate is expected to be 16kHz!')
        tfrec = make_tf_example(audio_serial, lbl)
        out_file.write(tfrec.SerializeToString())
        out_file.close()
        if i%50 == 0:        #if i == 50:
            print(f"Write {i} {lbl} to {out_file}")        #break
        i += 1
    print(f"------------------- finish {lbl} -------------------")

Write 50 clean to <tensorflow.python.lib.io.tf_record.TFRecordWriter object at 0x0000029F47691F88>
Write 100 clean to <tensorflow.python.lib.io.tf_record.TFRecordWriter object at 0x0000029F46638208>
Write 150 clean to <tensorflow.python.lib.io.tf_record.TFRecordWriter object at 0x0000029F476A6148>
Write 200 clean to <tensorflow.python.lib.io.tf_record.TFRecordWriter object at 0x0000029F4768F6C8>
Write 250 clean to <tensorflow.python.lib.io.tf_record.TFRecordWriter object at 0x0000029F4773E608>
Write 300 clean to <tensorflow.python.lib.io.tf_record.TFRecordWriter object at 0x0000029F47753988>
Write 350 clean to <tensorflow.python.lib.io.tf_record.TFRecordWriter object at 0x0000029F47763688>
Write 400 clean to <tensorflow.python.lib.io.tf_record.TFRecordWriter object at 0x0000029F465C48C8>
Write 450 clean to <tensorflow.python.lib.io.tf_record.TFRecordWriter object at 0x0000029F465F8448>
Write 500 clean to <tensorflow.python.lib.io.tf_record.TFRecordWriter object at 0x0000029F4781C948>
W